In [1]:
%run __init__.py
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import open3d as o3d

In [3]:
pnp_results = np.load('/home/konstantin/personal/CalibrationProject/calib_output/r_pnp_results.npy', allow_pickle=True).item()

In [4]:
r_pointclouds = np.load('/home/konstantin/personal/CalibrationProject/calib_output/r_pcd.npy', allow_pickle=True).item()

In [5]:
# del pnp_results[list(pnp_results.keys())[77]]

In [6]:
# del r_pointclouds[list(r_pointclouds.keys())[77]]

In [7]:
print(len(pnp_results))

118


In [8]:
print(len(r_pointclouds))

118


In [11]:
np.save('../calib_output/r_pnp_results.npy', pnp_results)
np.save('../calib_output/r_pcd.npy', r_pointclouds)

In [9]:
sum([i == j for i,j in zip(list(pnp_results.keys()), list(r_pointclouds.keys()))])

118

In [11]:
for key in pnp_results.keys():
    print(key)
    print(pnp_results[key][0]) # PnP transformation T: chessboard -> camera
    print(pnp_results[key][1].shape) # Transformed keypoints from chessboard: T @ kp_chessboard
    print(r_pointclouds[key].shape) # Segmented chessboard from Azure
    break

2012475554907
[[ 0.99127092  0.00252416  0.13181648 -0.01185215]
 [ 0.00531094  0.99824066 -0.0590541  -0.10442502]
 [-0.13173363  0.05923868  0.98951353  1.28062433]
 [ 0.          0.          0.          1.        ]]
(48, 3)
(21730, 3)


In [28]:
import mrob

In [296]:
#initialization
graph = mrob.FGraph()
W = np.array([1])
n1 = graph.add_node_pose_3d(mrob.geometry.SE3())
j=0
for key in pnp_results.keys():
    #decimate
    j=j+1
    if j%5!=0:
        continue
    pnp_chessboard_points = pnp_results[key][1]
    
    plane_pointcloud = r_pointclouds[key]
    
    plane_centroid = mrob.registration.estimate_centroid(plane_pointcloud)
    plane_normal = mrob.registration.estimate_normal(plane_pointcloud)
    #print(key, plane_centroid, plane_normal)
    
    if np.isnan(plane_centroid).any():
        print('Nan detected')
        #print(plane_pointcloud) # some are empty
        continue
    
    for i, p in enumerate(pnp_chessboard_points):
        if np.isnan(p).any():
            print('PnP Nan detected')
            continue
        graph.add_factor_1pose_point2plane(z_point_x = p,
                                           z_point_y = plane_centroid,
                                           z_normal_y = plane_normal,
                                           nodePoseId = n1,
                                           obsInf = W)
        

# solve the problem
graph.print(False)

Nan detected
Nan detected
Nan detected


In [297]:
graph.solve(mrob.GN)
result = graph.get_estimated_state()
print('From Smartphone to Azure', result[0])
print('inverse', mrob.geometry.SE3(result[0]).inv().T())
#graph.print(True)

From Smartphone to Azure [[ 0.99728918 -0.00370076 -0.07348877  0.2270376 ]
 [-0.00479579  0.99334178 -0.11510475 -0.02529519]
 [ 0.07342544  0.11514515  0.99063126 -0.04095365]
 [ 0.          0.          0.          1.        ]]
inverse [[ 0.99728918 -0.00479579  0.07342544 -0.22353641]
 [-0.00370076  0.99334178  0.11514515  0.03068259]
 [-0.07348877 -0.11510475  0.99063126  0.05434308]
 [ 0.          0.          0.          1.        ]]


In [313]:
#initialization
g2 = mrob.FGraph()
W = np.array([1])
n1 = g2.add_node_pose_3d(mrob.geometry.SE3())
j=0
for key in pnp_results.keys():
    #decimate
    j=j+1
    if j%6!=0:
        continue
    pnp_chessboard_points = pnp_results[key][1]
    
    plane_pointcloud = r_pointclouds[key]
    
    plane_centroid = mrob.registration.estimate_centroid(pnp_chessboard_points)
    plane_normal = mrob.registration.estimate_normal(pnp_chessboard_points)
    #print(key, plane_centroid, plane_normal)
    
    if np.isnan(plane_centroid).any():
        print('Nan detected')
        #print(plane_pointcloud) # some are empty
        continue
    
    for i, p in enumerate(plane_pointcloud):
        if np.isnan(p).any():
            print('Azure Nan detected')
            continue
        if i%100==0:
            g2.add_factor_1pose_point2plane(z_point_x = p,
                                           z_point_y = plane_centroid,
                                           z_normal_y = plane_normal,
                                           nodePoseId = n1,
                                           obsInf = W)
        

# solve the problem
g2.print(False)

In [314]:
g2.solve(mrob.LM,10)
result = g2.get_estimated_state()
print('From Azure to smartphone x100\n', result[0])
#graph.print(True)

From Azure to smartphone x100
 [[ 0.99696023 -0.00161908  0.07789527 -0.22216648]
 [-0.00699509  0.99388632  0.11018646  0.03609844]
 [-0.07759745 -0.1103964   0.99085381  0.0517138 ]
 [ 0.          0.          0.          1.        ]]


In [7]:
# x = o3d.geometry.PointCloud()
# x.points = o3d.utility.Vector3dVector(t_scene_points)
# #o3d.visualization.draw_geometries([x])
# print(len(pnp_results))

In [6]:
# pi = mrob.registration.estimate_plane(t_scene_points[0])
# print(pi)

In [309]:
# test visually
for key in pnp_results.keys():
    Y = pnp_results[key][1]
    X = r_pointclouds[key]
    pcx = o3d.geometry.PointCloud()
    pcx.points= o3d.utility.Vector3dVector(X)
    pcy = o3d.geometry.PointCloud()
    pcy.points= o3d.utility.Vector3dVector(Y)
    pcx.transform(result[0])
    o3d.visualization.draw_geometries([pcx,pcy])